# Day 18: Operation Order
As you look out the window and notice a heavily-forested continent slowly appear over the horizon, you are interrupted by the child sitting next to you. They're curious if you could help them with their math homework.

Unfortunately, it seems like this "math" [follows different rules](https://www.youtube.com/watch?v=3QtRK7Y2pPU&t=15) than you remember.

The homework (your puzzle input) consists of a series of expressions that consist of addition (`+`), multiplication (`*`), and parentheses (`(...)`). Just like normal math, parentheses indicate that the expression inside must be evaluated before it can be used by the surrounding expression. Addition still finds the sum of the numbers on both sides of the operator, and multiplication still finds the product.

However, the rules of **operator precedence** have changed. Rather than evaluating multiplication before addition, the operators have the **same precedence**, and are evaluated left-to-right regardless of the order in which they appear.

For example, the steps to evaluate the expression `1 + 2 * 3 + 4 * 5 + 6` are as follows:

    1 + 2 * 3 + 4 * 5 + 6
      3   * 3 + 4 * 5 + 6
          9   + 4 * 5 + 6
             13   * 5 + 6
                 65   + 6
                     71

Parentheses can override this order; for example, here is what happens if parentheses are added to form `1 + (2 * 3) + (4 * (5 + 6))`:

    1 + (2 * 3) + (4 * (5 + 6))
    1 +    6    + (4 * (5 + 6))
         7      + (4 * (5 + 6))
         7      + (4 *   11   )
         7      +     44
                51

Here are a few more examples:

* `2 * 3 + (4 * 5)` becomes **`26`**.
* `5 + (8 * 3 + 9 + 3 * 4 * 3)` becomes **`437`**.
* `5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))` becomes **`12240`**.
* `((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2` becomes **`13632`**.

Before you can help with the homework, you need to understand it yourself. **Evaluate the expression on each line of the homework; what is the sum of the resulting values?**

Your puzzle answer was `69490582260`.

In [1]:
input = """1 + (2 * 3) + (4 * (5 + 6))""".split('\n')

In [2]:
input = """2 * 3 + (4 * 5)""".split('\n')

In [3]:
input = """5 + (8 * 3 + 9 + 3 * 4 * 3)""".split('\n')

In [4]:
input = """5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))""".split('\n')

In [5]:
input = """((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2""".split('\n')

In [6]:
input = open('day.18.input.txt', 'r').read().split('\n')

In [7]:
import sys

from sly import Lexer, Parser

class AOCLexer(Lexer):
    tokens = {NUMBER}
    ignore = " \t"
    literals = {"+", "*", "(", ")"}

    @_(r"\d+")
    def NUMBER(self, t):
        t.value = int(t.value)
        return t
    
def AOCParserFactory(_precedence = (("left", "+", "*"),)):
    class AOCParser(Parser):
        tokens = AOCLexer.tokens
        precedence = _precedence

        @_("expr")
        def statement(self, p):
            return p.expr

        @_('expr "+" expr')
        def expr(self, p):
            return p.expr0 + p.expr1

        @_('expr "*" expr')
        def expr(self, p):
            return p.expr0 * p.expr1

        @_('"(" expr ")"')
        def expr(self, p):
            return p.expr

        @_("NUMBER")
        def expr(self, p):
            return p.NUMBER
        
    return AOCParser()

lexer = AOCLexer()
parser = AOCParserFactory()

sum([parser.parse(lexer.tokenize(line)) for line in input])

69490582260

## Part Two

You manage to answer the child's questions and they finish part 1 of their homework, but get stuck when they reach the next section: **advanced** math.

Now, addition and multiplication have **different** precedence levels, but they're not the ones you're familiar with. Instead, addition is evaluated **before** multiplication.

For example, the steps to evaluate the expression `1 + 2 * 3 + 4 * 5 + 6` are now as follows:

    1 + 2 * 3 + 4 * 5 + 6
      3   * 3 + 4 * 5 + 6
      3   *   7   * 5 + 6
      3   *   7   *  11
         21       *  11
             231

Here are the other examples from above:

* `1 + (2 * 3) + (4 * (5 + 6))` still becomes **`51`**.
* `2 * 3 + (4 * 5)` becomes **`46`**.
* `5 + (8 * 3 + 9 + 3 * 4 * 3)` becomes **`1445`**.
* `5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))` becomes **`669060`**.
* `((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2` becomes **`23340`**.

What do you get if you add up the results of evaluating the homework problems using these new rules?

Your puzzle answer was 362464596624526.

In [8]:
parser = AOCParserFactory(_precedence = (("left", "*"), ("left", "+"),))

sum([parser.parse(lexer.tokenize(line)) for line in input])

362464596624526